In [1]:
# First install geopy - for some reason I had issues installing with a pip command in the terminal
# This python code finally got it to install where it was needed
# !pip install geopy

In [2]:
# Initial imports
import pandas as pd
from geopy.geocoders import Nominatim

In [3]:
# Load the child care data set
file_path = "ChildCareSearchResults.csv"
care_df = pd.read_csv(file_path)
#print header to confirm import
care_df.head(2)

,Operation #,Agency Number,Operation/Caregiver Name,Address,City,State,Zip,County,Phone,Type,Status,Issue Date,Capacity,Email Address,Facility ID,Monitoring Frequency,Infant,Toddler,Preschool,School
0,1239246,NaN,"K"" Street Learning Center LLC""",19290 K ST,SOMERSET,TX,78069.0,BEXAR,830-429-1010,Licensed Center - Child Care Program,Full Permit,10/13/2011,82,sylvia@kstreetlearningcenter.com,773628,NaN,Y,Y,Y,Y
1,1687698,NaN,1.2..me Kidz,1222 BABCOCK RD,SAN ANTONIO,TX,78201.0,BEXAR,210-310-3990,Licensed Center - Child Care Program,Full Permit,3/21/2019,81,kidz1.2me@yahoo.com,1325841,NaN,Y,Y,Y,Y


In [4]:
# The Address column of data has whitespaces and would mess up stringing the address together, whitespaces removed
care_df['Address'] = care_df['Address'].str.strip()
care_df['City'] = care_df['City'].str.strip()
care_df['State'] = care_df['State'].str.strip()
care_df['County'] = care_df['County'].str.strip()

In [5]:
# The zip code imported with a decimal that had to be reformatted 
care_df['Zip'] = care_df['Zip'].apply('{0:g}'.format)

In [6]:
# Not the most efficient way to concantenate several columns to one column, but I got the data the way I needed it
care_df['Full_Address'] = care_df['Address'].map(str) + ', ' + care_df['City'].map(str) + ', ' + care_df['State'].map(str)+ ', ' + care_df['Zip'].map(str)
care_df.head(2)

,Operation #,Agency Number,Operation/Caregiver Name,Address,City,State,Zip,County,Phone,Type,...,Issue Date,Capacity,Email Address,Facility ID,Monitoring Frequency,Infant,Toddler,Preschool,School,Full_Address
0,1239246,NaN,"K"" Street Learning Center LLC""",19290 K ST,SOMERSET,TX,78069,BEXAR,830-429-1010,Licensed Center - Child Care Program,...,10/13/2011,82,sylvia@kstreetlearningcenter.com,773628,NaN,Y,Y,Y,Y,"19290 K ST, SOMERSET, TX, 78069"
1,1687698,NaN,1.2..me Kidz,1222 BABCOCK RD,SAN ANTONIO,TX,78201,BEXAR,210-310-3990,Licensed Center - Child Care Program,...,3/21/2019,81,kidz1.2me@yahoo.com,1325841,NaN,Y,Y,Y,Y,"1222 BABCOCK RD, SAN ANTONIO, TX, 78201"


In [7]:
#export the file to run through a lat-long program
care_df.to_excel('ChildCareSearchResults.xlsx')

## Rest of code is troubleshooting a free way to get lat-long using Geopy and Nominatim. 

In [8]:
# Test an example
locator = Nominatim(user_agent="insert your email")
location = locator.geocode("Champ de Mars, Paris, France")

In [9]:
# Success!  Nominatim works for small batches
print("Latitude = {}, Longitude = {}".format(location.latitude, location.longitude))

Latitude = 48.85614465, Longitude = 2.297820393322227


In [ ]:
## code works, however, some addresses with STE would not process for some unknown reason.

# from geopy.extra.rate_limiter import RateLimiter

# # The rate limiter function delays between geocoding calls
# geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
# # Create location column from the full address column
# care_df['location'] = care_df['Full_Address'].apply(geocode)

# # Create point column with longitude, latitude and altitude from location column (returns tuple)
# care_df['point'] = care_df['location'].apply(lambda loc: tuple(loc.point) if loc else None)
# # Split point column into latitude, longitude and altitude columns
# care_df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(care_df['point'].tolist(), index=care_df.index)

In [ ]:
care_df.head()

In [ ]:
# ended up using Geocodio to convert address to lat-longs, can do 2000 a day for free

## Unused code and sources 

In [ ]:
# from geopy import Nominatim
# import time

# geolocator = Nominatim(user_agent="mathgirl37@hotmail.com")

# with open("ChildCareSearchResults",'r') as fp:
#     for line in fp:
#         location = geolocator.geocode(line)
# print (location.latitude, location.longitude)
# time.sleep(1)
# fp.close()

In [ ]:
# # Source: https://towardsdatascience.com/geocode-with-python-161ec1e62b89
# # pip install geopy
# #locator = Nominatim(user_agent="childcare")
# # from geopy.extra.rate_limiter import RateLimiter

# # 1 - conveneint function to delay between geocoding calls
# # geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
# # 2- - create location column
# care_df['location'] = care_df['Full_Address'].apply(geocode)
# # 3 - create longitude, laatitude and altitude from location column (returns tuple)
# care_df['point'] = care_df['location'].apply(lambda loc: tuple(loc.point) if loc else None)
# # 4 - split point column into latitude, longitude and altitude columns
# care_df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(care_df['point'].tolist(), index=df.index)

In [ ]:
# from geopy.geocoders import Photon
# geolocator = Nominatim(user_agent="childcare")
# care_df['location'] = care_df['Full_Address'].apply(geocode)
# care_df['point'] = care_df['location'].apply(lambda loc: tuple(loc.point) if loc else None)


# location = geolocator.geocode("4200 Grim Ave, Waco, TX, 76710", timeout=None)
# print(location.address)
# Flatiron Building, 175, 5th Avenue, Flatiron, New York, NYC, New York, ...
# >>> print((location.latitude, location.longitude))
# (40.7410861, -73.9896297241625)
# >>> print(location.raw)
# {'place_id': '9167009604', 'type': 'attraction', ...}

In [ ]:
# care_df.head()

In [ ]:
# from geopy.geocoders import Photon
# geocoder = Nominatim(user_agent = 'your_app_name')
# from geopy.extra.rate_limiter import RateLimiter
# geocode = RateLimiter(geocoder.geocode, min_delay_seconds = 1,   return_value_on_exception = None) 
# # adding 1 second padding between calls

# # after initiating geocoder
# location = geocode(address)
# # returns location object with longitude, latitude and altitude instances
# (location.latitude, location.longitude)

In [ ]:
# #Importing the Nominatim geocoder class 
# from geopy.geocoders import Nominatim
 
# #address we need to geocode
# loc = 'Taj Mahal, Agra, Uttar Pradesh 282001'
 
# #making an instance of Nominatim class
# geolocator = Nominatim(user_agent="try_to_get_to_work")
 
# #applying geocode method to get the location
# location = geolocator.geocode(loc)
 
# #printing address and coordinates
# print(location.address)
# print((location.latitude, location.longitude))

In [ ]:
# Notes
# https://towardsdatascience.com/geocode-with-python-161ec1e62b89
# https://geopy.readthedocs.io/en/stable/#photon
# https://www.askpython.com/python/python-geopy-to-find-geocode-of-an-address
# https://www.geocod.io/
# https://geloky.com/geocoding/
# https://rigorousthemes.com/blog/15-best-tableau-dashboard-examples/
# https://public.tableau.com/app/profile/tableau.for.sales.analytics/viz/AccountManagement-360DegreeCustomerView/StrategicEnterpriseAccountProfile
# https://www.dataplusscience.com/SimpleOrgChart.html
# https://help.tableau.com/current/pro/desktop/en-us/calculations_calculatedfields_create.htm